# Unión de datos scrapeados, Dataframe final 

In [222]:
!ls -lrt

total 28328
-rw-r--r--  1 prealfer  staff   420474 31 ago 19:15 TFM_scraping.ipynb
-rw-r--r--  1 prealfer  staff        0  5 sep 23:51 README.md
-rw-r--r--  1 prealfer  staff        0  5 sep 23:51 README.txt
-rw-r--r--  1 prealfer  staff    15285 30 sep 20:02 maleteros
-rw-r--r--  1 prealfer  staff    16680 30 sep 20:06 Scrapeo-Maleteros.ipynb
-rw-r--r--  1 prealfer  staff  3877960  3 oct 21:29 Script scraping - Dataframe (Antiguo).ipynb
-rw-r--r--@ 1 prealfer  staff   721923  4 oct 23:12 segmentos.numbers
-rw-r--r--@ 1 prealfer  staff    11542  4 oct 23:14 segmentos.csv
-rw-r--r--@ 1 prealfer  staff      441  4 oct 23:21 total_segmento.csv
-rw-r--r--@ 1 prealfer  staff     1303  4 oct 23:24 variaciones_segmento.csv
-rw-r--r--  1 prealfer  staff    11786  4 oct 23:47 Limpieza datos publicos.ipynb
drwxr-xr-x  5 prealfer  staff      170 12 oct 23:59 FlaskApp
-rw-r--r--  1 prealfer  staff  2563080 17 oct 23:59 datos_crudo
-rw-r--r--  1 prealfer  staff  2551216 17 oct 23:59 datos_limpios
-

In [30]:
import pandas as pd
import numpy as np

In [31]:
pd.set_option('display.max_columns', None)

In [32]:
datos = pd.read_csv('Dataframe_limpio.csv',header=0)
datos = datos.drop('Unnamed: 0',axis=1)
maleteros = pd.read_csv('maleteros')
maleteros = maleteros.drop('Unnamed: 0',axis=1)
len(datos)

5932

## Unificamos en mayúsculas 

In [33]:
datos['marca'] = datos['marca'].apply(lambda x: x.upper().strip())
datos['modelo'] = datos['modelo'].apply(lambda x: x.upper().strip())
maleteros['Modelo'] = maleteros['Modelo'].apply(lambda x: x.upper())

### Creamos columna en 'datos' que contiene marca y modelo para cruzar con maleteros. A continuación, limpiamos de formatos y caracteres extraños y creamos una columna nueva con estos datos.

In [34]:
datos['mm']=datos['marca']+' '+datos['modelo']
modelos_data = datos['mm']
modelos_data = [mod.replace('NUEVO ','').replace('\xc3\xb3','O').replace('\xc3\xa9','E') for mod in modelos_data]
datos['MODELOS'] = modelos_data
datos.drop('mm', axis=1,inplace=True)

In [35]:
maleteros.columns = ['Marca','Modelo','Maletero']
modelos_maleteros = maleteros['Modelo']
modelos_maleteros = [mod.replace('NUEVO ','').replace(' 2017','').replace(' 2018','').replace('-BENZ','').replace('!','').replace('\xc5\xa0','S').replace('\xc3\xb3','O').replace('!','').replace('\xc3\xa9','E').replace('\xc3\xab','E') for mod in modelos_maleteros]
maleteros['MODELOS'] =  modelos_maleteros
maleteros.drop('Modelo',axis=1,inplace=True)
maleteros.rename(columns={'Marca': 'marca', 'marca-modelo': 'modelo', 'Maletero': 'maletero'}, inplace=True)

## Eliminamos los vehículos comerciales de nuestro Dataframe: 

### - No entran en el objetivo definido por negocio.
### - Sus características generarían 'outliers' (no del todo).

In [36]:
df = datos
len(df)

5932

In [37]:
df.marca.unique()

array(['ABARTH', 'AUDI', 'BMW', 'CITROEN', 'DACIA', 'DS', 'FIAT', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI', 'MORGAN',
       'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA',
       'SMART', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA',
       'VOLKSWAGEN', 'VOLVO'], dtype=object)

In [38]:
borrar_volkswagen = ['CADDY','CARAVELLE','CRAFTER','TRANSPORTER', 'UP COMERCIAL']
borrar_toyota = ['LAND CRUISER COMERCIAL', 'PROACE', 'PROACE VERSO']
borrar_seat = ['IBIZA COMERCIAL']
borrar_renault = ['KANGOO', 'MASTER', 'TRAFIC']
borrar_peugeot = ['BIPPER COMERCIAL', 'BOXER', 'EXPERT', 'PARTNER','TRAVELLER', 'TRAVELLER M1']
borrar_opel = ['COMBO', 'MOVANO', 'VIVARO']
borrar_nissan = ['EVALIA','NT400', 'NV200', 'NV300', 'NV400']
borrar_mercedes = ['CITAN', 'SPRINTER', 'VITO']
borrar_ford = ['CONNECT', 'TOURNEO CONNECT', 'TOURNEO COURIER',
       'TOURNEO CUSTOM', 'TRANSIT', 'TRANSIT COURIER', 'TRANSIT CUSTOM']
borrar_dacia = ['DOKKER', 'DOKKER COMERCIAL', 'LODGY']
borrar_citroen = ['BERLINGO','C3 COMERCIAL', 'JUMPER', 'JUMPY', 'NEMO', 'NEMO COMERCIAL', 'SPACETOURER']
borrar_fiat = ['DOBL\xc3\xb3', 'DUCATO',
       'FIORINO', 'FIORINO COMERCIAL', 'FULLBACK', 'TALENTO']

In [39]:
modelos_borrar = borrar_volkswagen + borrar_toyota + borrar_seat + borrar_renault + borrar_peugeot + borrar_opel + borrar_nissan + borrar_mercedes + borrar_ford + borrar_dacia + borrar_citroen + borrar_fiat

for modelo in modelos_borrar:
    df = df[df['modelo']!=modelo]

In [40]:
len(df)

4670

## Limpieza y codificación de las variables para poder ser utilizadas.


####  Eliminamos unidades, caracteres raros, espacios....

In [41]:
df['precio'] = df['precio'].apply(lambda x: x.replace('€','').replace('.','').strip())
df['puertas'] = df['puertas'].apply(lambda x: x.replace(' puertas','').strip())
df['cv'] = df['cv'].apply(lambda x: x.replace(' cv','').strip())
df['traccion'] = df['traccion'].apply(lambda x: x.replace('Tracción ','').strip())
df['n_marchas'] = df['n_marchas'].apply(lambda x: x.replace(' marchas','').strip())
df['co2'] = df['co2'].apply(lambda x: x.replace(' gr/km','').strip())
df['autonomia'] = df['autonomia'].apply(lambda x: x.replace(' km','').strip())
df['consumo'] = df['consumo'].apply(lambda x: x.replace(' l/100km','').strip())
df['cilindrada'] = df['cilindrada'].apply(lambda x: x.replace(' cc','').strip())
df['capacidad deposito'] = df['capacidad deposito'].apply(lambda x: x.replace(' lts.','').strip())
df['max_vel'] = df['max_vel'].apply(lambda x: x.replace(' km/h','').strip())
df['0_a_100'] = df['0_a_100'].apply(lambda x: x.replace(' s.','').strip())
df['rpm'] = df['rpm'].apply(lambda x: x.replace(' rpm','').strip())
df['par'] = df['par'].apply(lambda x: x.replace('Nm','').strip())
df['garantia'] = df['garantia'].apply(lambda x: x.replace(' meses','').replace('.','').strip())

#### Asignamos los tipos a las variables: Float a los numéricos para ser normalizados o categóricos para aplicarles una regla posteriormente

In [42]:
df['precio'] = df['precio'].astype(float)
df['puertas'] = df['puertas'].astype(float)
df['cv'] = df['cv'].astype(float)
df['traccion'] = df['traccion'].astype('category')
df['n_marchas'] = df['n_marchas'].astype(float)
df['co2'] = df['co2'].astype(float)
df['consumo'] = df['consumo'].astype(float)
df['cilindrada'] = df['cilindrada'].astype(float)
df['max_vel'] = df['max_vel'].astype(int)
df['capacidad deposito'] = df['capacidad deposito'].astype(float)
df['0_a_100'] = df['0_a_100'].astype(float)
df['rpm'] = df['rpm'].astype(float)
df['par'] = df['par'].astype(float)
df['garantia'] = df['garantia'].astype(float)
df['combustible'] = df['combustible'].astype('category')
df['cambio'] = df['cambio'].astype('category')

##  Sacamos del análisis columnas (variables) que no van a tomar parte de la aplicación.

In [43]:
cols_to_drop = ['airbags','marchas','control traccion','control estabilidad','alarma','cierre centralizado','tipo llave','peso','cierre automatico','start/stop','CD/mp3/dvd','cilindrada','max_vel','rpm','estrellas NCAP','par']

df = df.drop(cols_to_drop, axis=1)

## Metemos las capacidades de los maleteros

In [44]:
data = [140, 185, 505, 465, 530, 535, 292, 390, 480, 445, 530, 460, 440, 270, 310, 380, 350, 280, 364, 480, 130, 175, 488, 480, 470, 520, 540, 421, 550, 690, 680, 345, 225, 278, 860, 285, 490, 560, 356, 145, 280, 425, 380, 587, 610, 486, 680, 1975, 423, 243, 505, 512, 460, 430, 621, 547, 560]
s = ['124 SPIDER', '595', 'A4 ALLROAD', 'A5', 'A6 ALLROAD', 'A7', 'TT', 'SERIE 2', 'SERIE 3', 'SERIE 4', 'SERIE 5', 'SERIE 6', 'TIPO', 'KA','F-TYPE', 'CEED', 'CX-3', 'MAZDA2', 'MAZDA3', 'MAZDA6', 'MX-5', 'AMG GT', 'CLASE B', 'CLASE C', 'CLASE CLA', 'CLASE CLS', 'CLASE E', 'CLASE GLA', 'CLASE GLC', 'CLASE GLE', 'CLASE GLS', 'CLASE SL', 'CLASE SLC', 'MINI','MONTERO', 'CORSA','INSIGNIA', 'INSIGNIA COUNTRY TOURER', 'MOKKA', '911', 'BOXSTER', 'CAYMAN', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE', 'SCOUT', 'KORANDO', 'REXTON', 'RODIUS', 'TIVOLI', 'BRZ', 'FORESTER', 'OUTBACK', 'WRX STI', 'SX4 S-CROSS', 'LAND CRUISER', 'RAV-4', 'V90 CROSS COUNTRY']
maleteros_faltan = pd.Series(data, index=s)
maleteros2 = pd.DataFrame({'modelo':s,'maletero_2':data})

In [45]:
maleteros['maletero2'] = maleteros.maletero
maleteros.drop('maletero',axis=1,inplace=True)
maleteros.drop('marca',axis=1,inplace=True)
maleteros.head()

,MODELOS,maletero2
0,ALFA ROMEO MITO,270
1,ALFA ROMEO 4C,110
2,ALFA ROMEO GIULIETTA,350
3,ALFA ROMEO GIULIA,480
4,ALFA ROMEO STELVIO,525


In [46]:
df_maleteros_1 = pd.merge(df,maleteros,how='outer')
df_maleteros_1.drop(df_maleteros_1[df_maleteros_1['marca'].isnull()].index,inplace=True)

In [47]:
df_maleteros_2 = pd.merge(df_maleteros_1,maleteros2,how='outer')

In [48]:
def suma_maleteros(row):
    if(np.isnan(row['maletero2'])):
       return row['maletero_2']       
    else:
       return row['maletero2']
       


In [49]:
df_maleteros_2['maletero'] = df_maleteros_2.apply(suma_maleteros, axis=1)
df_maleteros_2.drop(['maletero2','maletero_2'], axis=1, inplace=True)
df_maleteros_2['maletero'] = df_maleteros_2['maletero'].astype(float)

### Creamos la variable tipo de marca, para identificar marcas generalistas, premiums y gran lujo 

In [50]:
df2 = df_maleteros_2

In [51]:
marcas = ['ABARTH', 'FIAT', 'AUDI', 'BMW', 'CITROEN', 'DACIA', 'DS', 'FORD',
       'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS',
       'MASERATI', 'MAZDA', 'MERCEDES', 'MINI', 'MITSUBISHI', 'NISSAN',
       'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SEAT', 'SKODA', 'SMART',
       'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN',
       'VOLVO']

In [52]:
tipos_marca = [0,0,2,2,0,0,0,0,0,0,2,2,0,0,2,3,0,2,2,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,2]

In [53]:
marcas_type = pd.DataFrame({'marca':marcas,'tipo_marca':tipos_marca})

In [54]:
df_marcas = pd.merge(df2,marcas_type,how='outer')

### Creamos la variable segmento. Indica el segmento comercial al que pertenece el vehículo 

In [55]:
df3 = df_marcas

In [56]:
segmentos = pd.read_csv('segmentos.csv',sep=';')
segmentos['modelo'] = segmentos['Modelo']
segmentos['segmento1'] = segmentos['Segmento']
segmentos.drop(['Marca','Segmento','Modelo'], axis=1,inplace=True)

In [57]:
mod = ['595', '500L', '500X', 'PUNTO', 'A4 ALLROAD', 'A6 ALLROAD', 'I3',
       'I8', 'C-ELYS\xc3\xa9E', 'C3 AIRCROSS', 'DS4 CROSSBACK', 'C-MAX',
       'CR-V', 'HR-V', 'I10', 'I20', 'I20 ACTIVE', 'I30', 'I40', 'IX20',
       'Q60', 'E-PACE', 'F-TYPE CONVERTIBLE', 'LC', 'GRANTURISMO',
       'MAZDA2', 'MAZDA3', 'MAZDA6', 'AMG GT', 'COUNTRYMAN', 'I-MIEV',
       '370Z', 'CROSSLAND X', 'GRANDLAND X', 'INSIGNIA COUNTRY TOURER',
       '508 RXH', 'ION', 'KOLEOS', 'M\xc3\xa9GANE', 'SC\xc3\xa9NIC',
       'TWIZY', 'LE\xc3\xb3N', 'LE\xc3\xb3N X-PERIENCE', 'MII', 'KODIAQ',
       'SCOUT', 'BRZ', 'WRX STI', 'IGNIS', 'GT86', 'LAND CRUISER',
       'ARTEON', 'GOLF ALLTRACK', 'PASSAT ALLTRACK', 'UP',
       'V90 CROSS COUNTRY', 'XC60', 'XC90']

seg = ['Superutilitario', 'Monovolumen polivalente', 'TTE Pequeño', 'Utilitario/Polivalente', 'Familiar', 'Familiar', 'Monovolumen polivalente',
       'Deportivo alto', 'Berlina', 'Monovolumen polivalente', 'Compacto', 'Monovolumen compacto',
       'TTE Familiar', 'TTE Compacto', 'Superutilitario', 'Utilitario/Polivalente', 'Utilitario/Polivalente', 'Compacto', 'Familiar', 'Monovolumen polivalente',
       'Deportivo alto', 'TTE Compacto', 'Deportivo alto', 'Deportivo alto', 'Gran lujo',
       'Utilitario/Polivalente', 'Compacto', 'Berlina', 'Deportivo alto', 'TTE Peque\xc3\xb1o', 'Superutilitario',
       'Deportivo bajo', 'TTE Pequeño', 'TTE Compacto', 'Familiar',
       'Familiar', 'Superutilitario', 'TTE Familiar', 'Compacto', 'Monovolumen familiar',
       'Superutilitario', 'Compacto', 'Familiar', 'Superutilitario', 'TTE Familiar',
       'Familiar', 'Deportivo bajo', 'Compacto', 'Superutilitario', 'Deportivo bajo', 'TTE Superior',
       'Berlina', 'Familiar', 'Familiar', 'Superutilitario',
       'Familiar', 'TTE Familiar', 'TTE Superior']

segmentos2 = pd.DataFrame({'modelo':mod, 'segmento2':seg})

In [58]:
df_segmentos = pd.merge(df3,segmentos,how='outer')
df_segmentos.drop(df_segmentos[df_segmentos['marca'].isnull()].index,inplace=True)

In [59]:
df_segmentos_2 = pd.merge(df_segmentos,segmentos2,how='outer')
df_segmentos_2.drop(df_segmentos_2[df_segmentos_2['marca'].isnull()].index,inplace=True)

In [60]:
df_segmentos_2['segmento'] = df_segmentos_2.segmento1.combine_first(df_segmentos_2.segmento2)
df_segmentos_2.drop(['segmento1','segmento2'],axis=1,inplace=True)

### Eliminamos vehículos especiales como el Renault Twizy 

In [61]:
df4 = df_segmentos_2

In [62]:
row_twizy = df4[df4.cv==17].index
df4.drop(row_twizy, inplace=True)

In [63]:
def longitud(carroceria):
    return carroceria[:4]

In [64]:
df4['longitud'] = df4['carroceria'].apply(longitud)
df4.drop('carroceria',axis=1,inplace=True)
df4['longitud'] = df4['longitud'].astype(float)
df4['Segmento'] = df4['segmento'].astype('category')
df4['marca'] = df4['marca'].astype('category')

### Agrupamos tipos de tracción: ['trasera', 'delantera', 'total'] 

In [65]:
tracciones_totales = ['Todo terreno tracción total permanente', 'total permanente', 'Todo terreno tracción total conectable', 'total conectable']

def traccion(row):
    if(str.strip(row['traccion']) in tracciones_totales):
        return 'total'
    else:
        return row['traccion']
    

In [66]:
df4['tipo_traccion'] = df4.apply(traccion,axis=1)
df4['traccion'] = df4['tipo_traccion']
df4.drop('tipo_traccion',axis=1,inplace=True)
df4['traccion'].unique()

array(['trasera', 'delantera', 'total'], dtype=object)

## Preparar el Dataframe para el cálculo de distancias 

#### Eliminamos las columnas categóricas que sirven para identificar al vehículo, pero que no toman parte en el cálculo de distancias sobre ellas 

In [67]:
cols_descriptivas = ['marca','MODELOS']
df4.drop(cols_descriptivas, axis=1,inplace=True)
df4['cambio'] = df4['cambio'].apply(str.strip)

### Agrupamos los tipos de cambio en Automático, automatizado y manual

In [68]:
cambios_automaticos = ['Automática secuencial', 'Directo. sin caja de cambios', 'Automática continua. secuencial', 'Automática', 'Automática continua']

In [69]:
def orden_cambios(row):
    if row['cambio'] in cambios_automaticos:
        return 'automatico'
    elif (row['cambio']=='Manual automatizada'):
        return row['cambio']
    else:
        return 'manual'

In [70]:
df4['cambio_marchas'] = df4.apply(orden_cambios,axis=1)
df4.cambio = df4.cambio_marchas
df4.drop('cambio_marchas', axis=1,inplace=True)

### Codificamos el tipo de tracción Delantera->1; Trasera 0.5; Total->0


In [72]:
def cod_traccion(row):
    if(row['traccion'].strip()=='delantera'):
        return 0
    elif(row['traccion'].strip()=='trasera'):
        return 0.5
    else:
        return 1

In [73]:
df4['cod_tra'] = df4.apply(cod_traccion,axis=1)
df4['traccion'] = df4['cod_tra']
df4['traccion'] = df4['traccion'].astype(float)
df4.drop('cod_tra',axis=1,inplace=True)

### Codificamos combustible:

#### Vamos a codificar los tipos de combustible (realmente, energía) con las que trabaja el veícuhlo. Tratamos de ordenar por limpieza medioambiental, comenzando por eléctricos puros hasta llegar a los vehículos diésel.

In [74]:
hibridos = ['Diesel y corriente eléctrica', 'Gasolina y corriente eléctrica']

def cod_comb(row):
    if(row['combustible']=='Corriente eléctrica'):
        return 0
    elif(row['combustible'] in hibridos):
        return 0.1
    elif(row['combustible']=='Gasolina/gas'):
        return 0.25
    elif(row['combustible']=='Gasolina'):
        return 0.85
    else:
        return 1

In [75]:
df4.combustible = df4.combustible.apply(str.strip)
df4['cod_comb'] = df4.apply(cod_comb,axis=1)
df4['combustible'] = df4['cod_comb']
df4.drop('cod_comb',axis=1,inplace=True)

### Codificamos el cambio de marchas 

In [76]:
def cod_cambio(row):
    if(row['cambio']=='manual'):
        return 0
    elif(row['cambio']=='Manual automatizada'):
        return 0.9
    else:
        return 1

In [77]:
df4['cod_camb'] = df4.apply(cod_cambio,axis=1)
df4['cambio'] = df4['cod_camb']
df4.drop('cod_camb',axis=1,inplace=True)

### Codificamos el segmento al que pertenece el vehículo, aunque en el futuro quizás sólo tome parte para filtrar los requisitos del usuario. 

In [78]:
def cod_seg(row):
    if(row['Segmento']=='Superutilitario'):
        return 0
    elif(row['Segmento']=='Utilitario/Polivalente'):
        return 4
    elif(row['Segmento']=='Compacto'):
        return 10
    elif(row['Segmento']=='TTE Pequeño'):
        return 12
    elif(row['Segmento']=='TTE Compacto'):
        return 15
    elif(row['Segmento']=='Berlina'):
        return 25
    elif(row['Segmento']=='Familiar'):
        return 30
    elif(row['Segmento']=='Monovolumen polivalente'):
        return 40
    elif(row['Segmento']=='Monovolumen compacto'):
        return 43
    elif(row['Segmento']=='Monovolumen familiar'):
        return 45
    elif(row['Segmento']=='TTE Familiar'):
        return 60
    elif(row['Segmento']=='TTE Superior'):
        return 65
    elif(row['Segmento']=='Gran lujo'):
        return 85
    elif(row['Segmento']=='Deportivo alto'):
        return 95
    elif(row['Segmento']=='Deportivo bajo'):
        return 100

In [79]:
df4['cod_seg'] = df4.apply(cod_seg,axis=1)
df4['Segmento'] = df4['cod_seg']
df4.drop('cod_seg',axis=1,inplace=True)

#### Cambiamos el formato de las autonomías -> eliminamos los puntos que delimitan los millares

In [80]:
def cifras(item):
    if(item[1]=='.'):
        return (item[0]+item[2:]).strip()
    else:
        return item.strip()

In [81]:
df4['autonomia2'] = df4['autonomia'].apply(cifras)
df4['autonomia'] = df4['autonomia2']
df4.drop('autonomia2',axis=1, inplace=True)

#### Seteamos a tipo float las variables que falten, para su normalización 

In [82]:
df4['Segmento'] = df4['Segmento'].astype(float)
df4['puertas'] = df4['puertas'].astype(float)
df4['autonomia'] = df4['autonomia'].astype(float)


In [83]:
df4 = df4.drop_duplicates()
df4 = df4.dropna()

#### Sacamos las columnas identificativas de los vehículos ya que no van a ser normalizadas y utilizadas en el cálculo de distancias. .

In [84]:
datos_sin_normalizar = df4.drop(['modelo','version','segmento'],axis=1)


### Función que normaliza los datos conforme al mínimo. 

In [85]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result   

In [86]:
normalizada = normalize(datos_sin_normalizar)

# Calcular distancias euclídeas

In [87]:
from scipy.spatial import distance

In [88]:
array_norm = normalizada.values
df_listado = df4[['modelo','version','precio','segmento']]

In [89]:
def recomienda_50(index):

    recorrido = range(len(normalizada))
    distancias = []

    for i in recorrido:
        distancias.append(distance.euclidean(array_norm[index], array_norm[i]))
        
    df_dist = pd.DataFrame({'dist':distancias, 'ind':recorrido})
    
    return df_dist.sort_values('dist', ascending=True).head(50).index

In [90]:
df_listado.to_csv('listado.csv',encoding='utf-8')
normalizada.to_csv('normalizado.csv',encoding='utf-8')